<a href="https://colab.research.google.com/github/Jakub-MFP/My_FIRE_Project/blob/master/portfolio_management/cashposition_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Objective**

**Problem**

Lots of resources advocate for investors to have between 10% to 30% of their portfolio in cash so we can seize upon market opportunities. The problem is that it is never explained when we should have 30% or 20% or 10% cash position, what factors determine how much cash we should have at any given time?


**Question**

What is the optimal cash position to keep in our portfolio at anytime for the past 20 years. We will assume that our portfolio only has one asset and it is the ETF SPY. Which is an ETF that tracks the SP500 Index and is a good overall indicator for the status of the "market"


**Hypothesis**

I think we will figure out that just investing 100% of the money into a portfolio during our regular deposit cycles and keeping no cash position will under-perform having a cash position at all times. 

*For example*

* If the SPY is up, we keep 30% of our portfolio in cash
* IF SPY is down 10% from ATH (all time high) than we keep 20% of portfolio in cash
* IF SPY is down 20% from ATH than we keep 10% of portfolio in cash
* IF SPY is down 30% from ATH than we keep 5% of portfolio in cash 

I believe that having a cash position on the side and investing more when the market is down will yield a higher ROI and CAGR over the same 20 year period. 

If SPY just regular investments returned a 9% CAGR, than I believe doing something as simple as just holding some cash will return 12% CAGR+. Something significant like 30-40% higher returns than otherwise possible. 

This will be tested not just on SPY but a number of stocks and ETFs with the eventually goal of doing this  with multiple stocks in a portfolio and using the efficient frontier to determine how much we allocate to any stock when the market trigger is in effect to reduce or increase our cash position. 

**The Procedure**
1. We run the experiment to see how much 10,000 in 2000 would yield by end of Dec 2019 with no additional investments. We want to calcualte the CAGR or Compound Annual Growth Rate
2. We run another experiment to see how much 10,000 in 2000 would yield by end of Dec 2019 with a 1,000 monthly deposit. Does it change the CAGR? 
3. Finally we will run a combinatorics trial 

   1. Start with 10,000
        1. Deposit each month 1,000 into portfolio
        2. Adjust the invested / cash ratio based on the market performance
            1. If the SPY is all time high than keep 70% invested and 30% in cash
            2. If the SPY drops 10% from ATM than keep 80% invested and 20% cash
            3. If SPY drops 20% from ATM than keep 90% invested and 10% cash
            4. If SPY drops 30% from ATM than keep 95% invested and 5% cash
        3. Do every combination of cash allocation based on how much market drops between -1% and -50%
        4. Calcualte the CAGR for each combination

Daily Progress Update 
https://myfireproject.com/topic/301-portfolio-management-research/


# **Initial Setup**

Installs Alpha Vantage and imports required modules

**Install Alpha Vantage API**

In [ ]:
# Install Alpha Vantage
!pip install pip install alpha_vantage

**Import All The Modules You Need**

In [ ]:
import pandas as pd
from pandas import DataFrame
import json
import requests
import sqlite3
import time
import datetime
from datetime import datetime as dt
datetime.datetime.strptime
from alpha_vantage.timeseries import TimeSeries
import numpy as np
import matplotlib.pyplot as plt 
import sys
import math

# **Alpaha Vantage API**

This will create a pandas dataframe of all historical prices

https://www.alphavantage.co/documentation/



Assign Stock Ticker and API Key

In [ ]:
stock_ticker = 'SPY'
   
    # Update this for your own API key
api_key = open('/content/drive/My Drive/Colab Notebooks/key').read()
#api_key = '4545sdsad5s4dsd'

Run API Request to get stock data

    # Date / Open / High / Low / Close / Adjusted Close / Volume / Dividend / Split

In [ ]:
ts = TimeSeries (key=api_key, output_format = "pandas")
data_daily, meta_data = ts.get_daily_adjusted(symbol=stock_ticker, outputsize ='full')

Assign metrics from dataframe to individual variables

In [ ]:
    # data_daily['column name'][row number] 
# stock_price_open = data_daily['1. open'][0]
# stock_price_close = data_daily['5. adjusted close'][0]
# dividend_amount = data_daily['7. dividend amount'][0]

#**Procedure**
Investment Period = Jan 1, 2000 to Dec 31, 2019
1. ***ROI*** and ***CAGR*** of Investing `$10,000` and just allocating `100%` Position
2. ***ROI*** and ***CAGR*** of Investing `$10,000` and depositing `$1,000` each month and allocating `100%` position 
3. ***ROI*** and ***CAGR*** of Investing `$10,000` and depositing `$1,000` each month and allocating a `variable` position using `market_status`

## Step 1. `$10,000` and `100% Position` 

1. The goal is to get the dataframe to start on `start_date` and end on `end_date`
2. Grab the `adjusted_close_price` for `start_date` or the next trading day therafter. 
3. `starting_shares` = `initial_investment` / `adjusted_close_price` for the `start_date`
4. Grab the `adjusted_close_price` for the `end_date` 
5. `step1_final_value` = `starting_shares` * `adjusted_close_price`
6. `step1_roi` = (`step1_final_value` - `initial_investment`) / `initial_investment`
7. `step7_cagr` = round(((math.log(`step1_final_value`/`initial_investment`)) / (math.log(1+`number_of_years`))),2)

In [60]:
start_year = 2000
start_month = 1
start_day =  1

end_year = 2019
end_month = 12
end_day = 31

# number_of_years = end_date - start_date #should be 20 years ish
number_of_years = 20

    # Step 1


initial_investment = 10000
monthly_deposit = 1000

    # Start Price
date_filter_start = data_daily.filter(like = '2000-01-03', axis = 0) 
start_adjusted_close_price = date_filter_start['5. adjusted close'][0]
starting_shares = initial_investment / start_adjusted_close_price
# start_reported_date = date_filter['date'][-1] #not working

    # End Price
date_filter_end = data_daily.filter(like = '2019-12-31', axis = 0)
end_adjusted_close_price = date_filter_end['5. adjusted close'][0]

# end_reported_date = date_filter['date'][0] #not working

    # Final Math
step1_final_value = round((starting_shares * end_adjusted_close_price), 2)
step1_profit = round((step1_final_value - initial_investment), 2)
step1_roi = round((((step1_final_value - initial_investment) / initial_investment)* 100), 2)
step1_cagr = round((((step1_final_value / initial_investment) ** (1 / (number_of_years - 1)) - 1)* 100), 2)

    # Print out results
#print("Starting Date : {} ".format(start_reported_date))
print("Start Share Price : ${} ".format(start_adjusted_close_price))
print("Starting Shares : {} ".format(starting_shares))
print("")
#print("Ending Date : {} ".format(start_reported_date))
print("Ending Share Price : ${} ".format(end_adjusted_close_price))
print("")
print("Total Deposits : ${} ".format(initial_investment))
print("Portfolio Value : ${} ".format(step1_final_value))
print("Investment Profit : ${} ".format(step1_profit))
print("ROI : {}% ".format(step1_roi))
print("CAGR : {}% ".format(step1_cagr))

Start Share Price : $98.5115 
Starting Shares : 101.51099110256163 

Ending Share Price : $317.2003 

Total Deposits : $10000 
Portfolio Value : $32199.32 
Investment Profit : $22199.32 
ROI : 221.99% 
CAGR : 6.35% 


Testing out CAGR

In [57]:
end_value = 32199
start_value = 10000
num_periods = 20

CAGR = (end_value / start_value) ** (1 / (num_periods - 1)) - 1

print(CAGR)


0.06347808908458075


In [ ]:
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

testing out date filters

start date

In [43]:
date_filter_start = data_daily.filter(like='2000-01-03', axis = 0) 
start_adjusted_close_price = date_filter_start['5. adjusted close'][0]
starting_shares = initial_investment / start_adjusted_close_price
print(start_adjusted_close_price)

98.5115


end date

In [47]:
date_filter_end = data_daily.filter(like='2019-12-31', axis = 0)
end_adjusted_close_price = date_filter_end['5. adjusted close'][0]
print(start_adjusted_close_price)

317.2003


In [ ]:
data_daily.filter(like='2000-01-03', axis = 0)
# start_adjusted_close_price = date_filter['5. adjusted close'][-1]
# print(start_adjusted_close_price)

In [34]:
data_daily.filter(like='2019-12-31', axis = 0)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2019-12-31,320.53,322.13,320.15,321.86,317.2003,57059907.0,0.0,1.0


In [ ]:
test = date_filter['date'][-1]
print(test)

In [ ]:
print(date_filter)

## Step 2. `$10,000` + `$1,000` Monthly and `100% Position` 

In [ ]:
# start_date = '2000-01-01'
# end_date = '2019-12-31'
# number_of_years = float(end_date - start_date) #should be 20 years ish
number_of_years = 20

initial_investment = 10000
monthly_deposit = 1000


    # step 1
date_filter = data_daily # range between start_date and end_date to create new dataframe

    # step 2 - Get Starting current_balance
start_current_balance = initial_investment

    # step 3 - get starting current_shares
start_adjusted_close_price = date_filter['5. adjusted close'][0]
start_current_shares = start_current_balance / start_adjusted_close_price

    # Settings for loop
current_balance = 0
current_shares = 0
current_date = 0
total_deposits = 0
deposit_dates = {}  # date , investment amount 

deposit_dates.append(start_date, int(initial_investment)) # add starting balance to deposit_dates_list

    # step 4 - Itterate trough all the rows in the dataframe and buy shares if it is a new month
for index, row in date_filter.iterrows():
  
    current_date = index # set current index date row to current_date

    if current_date #is new month ():
        current_balance =+ monthly_deposit # add $1000 to current_balance
        current_shares =+ (current_balance / row['5. adjusted close'] ) # figure out how many new shares we are buying 
        total_deposits =+ 1 # add to total_deposits that we made a new deposit
        deposit_dates.append(current_date, int(monthly_deposit)) #add current date for each deposit to the dic 
    elif:
        continue

    # Step 6. figure out average age of deposits
age_ratio = []

for date, deposit in deposit_dates.items(): # loop trough all the deposits in dictironary and figure out age ratio for each deposit
    ratio = float(((end_date - date)/365) * deposit)
    age_ratio.append(ratio)


    # End Price
end_adjusted_close_price = date_filter['5. adjusted close'][-1] #get the ending adjusted_close price

    # Final Math
step2_total_deposits = int((total_deposits * monthly_deposit) + initial_investment)
step2_final_value = round((current_shares * end_adjusted_close_price), 2)
step2_profit = round((step2_final_value - step2_total_deposits, 2)
step2_roi = round((step2_profit / step2_total_deposits), 2)
step2_age = round((float(sum(age_ratio) / step2_total_deposits)),2)
step2_cagr = round(((math.log(step2_final_value/step2_total_deposits)) / (math.log(1+step2_age))),2)

    #print results
print("Total Deposits : {} ".format(step2_total_deposits))
print("Portfolio Value : {} ".format(step2_final_value))
print("Investment Profit : {} ".format(step2_profit))
print("ROI : {} ".format(step2_roi))
print("CAGR : {} ".format(step2_cagr))



## Step 3. `$10,000` + `$1,000` Monthly and `Variable Position` 

In [ ]:
start_date = '2000-01-01'
end_date = '2019-12-31'
number_of_years = end_date - start_date #should be 20 years ish


initial_investment = 10000
monthly_deposit = 1000


    # Step 1 > Set date range between start_date and end_date to create new dataframe
date_filter = data_daily #


    # Step 2 > Starting settings
current_balance = 10000
current_shares = 0
current_date = 0
portfolio_value = 0

total_deposits_count = 0
total_shares_purchases_count = 0

portfolio_deposits = {} #log every deposit in dictionary 
portfolio_trades = {} #log every trade in dictrionary
market_statuses = {} #log every market status for each day


    # Main Loop
for index, row in date_filter.iterrows():

    current_date = index # set current index date row to current_date

        # Determine if its a date we deposit or not
    if current_date # is new month ():
        current_balance =+ monthly_deposit # add $1000 to current_balance
        total_deposits_count =+ 1 # add to total_deposits that we made a new deposit
        portfolio_deposits.append(current_date, int(monthly_deposit)) #add current date for each deposit to the dic
    else:
        continue


        # Figure out what the market status is, if it is down or up compared to all time high
    current_market_price = data_daily['5. adjusted close']
    market_all_time_high = float(max(portfolio_trades)[1]) # highest value in the dictionary 
    current_market_change = float(current_market_price - market_all_time_high) # ex -50
    current_market_rating = float(current_market_change / market_all_time_high) # ex -0.33
    market_satuses.append(current_date, current_market_price, market_all_time_high, current_market_change, current_market_rating)


        # How much cash do we need to kee on hand based on the market status
            # https://www.tutorialspoint.com/python/comparison_operators_example.htm

    if (current_market_status > 0):  #greater than 0
        current_cash_required_equity = 0.30
    elif (current_market_status > -0.05): #less than 5%
        current_cash_required_equity = 0.25
    elif (current_market_status > -0.10): #less than 10%
        current_cash_required_equity = 0.20 
    elif (current_market_status > -0.15): #less than 15%
        current_cash_required_equity = 0.15
    elif (current_market_status > -0.20): #less than 20%
        current_cash_required_equity = 0.10
    elif (current_market_status > -0.25): #less than 25%
        current_cash_required_equity = 0.05
    elif (current_market_status > -0.30): #less than 30%
        current_cash_required_equity = 0


        # How many shares do we need to buy if any based on market status
    portfolio_value = float((current_balance + (current_shares * current_market_price)))
    current_cash_required = float(current_cash_required_equity * portfolio_value)
    if (current_cash_required < current_balance): 
        cash_to_purchase_shares = float(current_balance - current_cash_required)
        current_shares =+ (cash_to_purchase_shares / row['5. adjusted close'] ) # figure out how many new shares we are buying
        current_balance =- (cash_to_purchase_shares) # adjust current balance for shares purchased
        total_shares_purchases_count =+ 1 # add 1 to total count
        portfolio_trades.append(current_date, cash_to_purchase_shares, current_shares, current_balance)
    else ():
        continue
  

    # Figuring out Age Ratio
age_ratio = []

for date, portfolio_depositss in portfolio_deposits.items(): # loop trough all the deposits in dictironary and figure out age ratio for each deposit
    ratio = float(((end_date - date)/365) * deposit)
    age_ratio.append(ratio)


    # Final Math
step3_total_deposits = int((total_deposits_count * monthly_deposit) + initial_investment)
step3_final_value = round(portfolio_value, 2)
step3_profit = round((step3_final_value - step3_total_deposits, 2)
step3_roi = round(((step3_profit / step3_total_deposits), 2)
step3_age = round((float(sum(age_ratio) / step3_total_deposits)),2)
step3_cagr = round(((math.log(step3_final_value/step2_total_deposits)) / (math.log(1+step3_age))),2)


    # Print Results
print("Total Deposits : {} ".format(step3_total_deposits))
print("Portfolio Value : {} ".format(step3_final_value))
print("Investment Profit : {} ".format(step3_profit))
print("ROI : {} ".format(step3_roi))
print("CAGR : {} ".format(step3_cagr))

# **Reasearch Notes**

Testing out elif loops

In [ ]:
current_cash_required = 0
current_market_status = -0.09

if (current_market_status > 0):  #greater than 0
    current_cash_required_equity = 0.30
elif (current_market_status > -0.05): #less than 5%
    current_cash_required_equity = 0.25
elif (current_market_status > -0.10): #less than 10%
    current_cash_required_equity = 0.20 
elif (current_market_status > -0.15): #less than 15%
    current_cash_required_equity = 0.15
elif (current_market_status > -0.20): #less than 20%
    current_cash_required_equity = 0.10
elif (current_market_status > -0.25): #less than 25%
    current_cash_required_equity = 0.05
elif (current_market_status > -0.30): #less than 30%
    current_cash_required_equity = 0

print(float(current_cash_required_equity))

0.2


## **Possible structure for the loop**

In [ ]:
current_cash = 10000
deposit_amount = 1000
deposit_frequency = 5 #every 7 days
deposit_dates = [] # this list is of dates that a deposit will execute one 
    # we need to create a loop that will append 7 week to the current date and generate a list of all mondays
    # If monday is closed for trading, than we will use next avaiable trading day
    # than it resets back to original 
    # print out the generated deposit dates to test if this works 



count_trades = 0 #number of times 
count_deposits = 0 #count number of deposits
count_cash_deposited = 0 # count total value of cash that was depoisited 
count_profit = 0 #total profit of portfolio

current_shares_count = 0 # total number of shares owend
current_stock_price = 0 # current stock market price



for i in df.index:
     
    current_trading_date = # the next date in the loop trough the dataframe
    current_stock_price = # the next stock price in  the loop troough the dataframe
    current_market_all_time_high = # this needs to be an if statement
        # if current price is above market high than set new current_market_all_time_high
        # if its not ,t han the current one stays 
    current_market_change = current_stock_price - current_market_all_time_high

    # Setting market status
    current_market_status = current_market_change - current_market_all_time_high
    
    current_cash_required = # loop based on current marekt status
        if current_market_status > 0 than current_cash_required = 0.30
        elif current_market_status < 0  >=-5 than current_cash_required = 0.25
        elif current_market_status < -5  >=-10 than current_cash_required = 0.20
        elif current_market_status < -15  >=-15 than current_cash_required = 0.10


    # now that we know market status, we need see if we can deposit or not
    if current_trading_date == a date inside the lsit deposit_dates:
        current_cash =+ deposit_amount # we deposit the amoutn set into current cash
    elif continue 

    current_cash_equity = current_cash / current_portfolio_value
    
    # now we need to figure out if we need to buy shares or not
    if current_cash_equity > required_cash_equity
        stock_purchase_cash = current_cash_equity - required_cash_equity
        total_shares_purchased = stock_purchase_cash / current_stock_price
        current_shares_count =+ total_shares_purchased

    # Now we set current portfolio value 
    current_portfolio_value = (current_shares_count * current_stock_price) + current_cash

    # now we add all of this data to new dataframe that will track the results for each day
        # save date, and all the metrics that where set bc of this 

# Now at the end print out the to tal profit, total tradess, ect 









## **Itteration Test**

In [ ]:
    # Creating a list of all days thaht will have a trade executed

matches = []
for index, row in df.iterrows():
    item = index
for match in date_list:
    if index in match:
        matches.append(match)

    #print(index, row['1. open'])
print(matches)



##  **Interacting With DataFrame**

For each row in the dataframe, displaying the date and the open price

In [ ]:
for index, row in data_daily.iterrows():
    print(index, row['1. open'])

In [ ]:
for index, row in data_daily.iterrows():
    

print out all columns names in dataframe

In [ ]:
for (columnName, columnData) in data_daily.iteritems():
   print('Colunm Name : ', columnName)
   print('Column Contents : ', columnData.values)

Plotting Charts

In [ ]:
data_daily['4. close'].plot()
plt.title('Stock chart')
plt.show()

Filtering Date Ranges in the pandas dataframe 

In [ ]:
start_date = "2019-1-1"
end_date = "2019-1-31"

after_start_date = data_daily['index'] >= start_date
before_end_date = data_daily['index'] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = df.loc[between_two_dates]

print(filtered_dates)

In [ ]:
start_date = '03-01-1996'
end_date = '06-01-1997'

mask = (df['date'] > start_date) & (df['date'] <= end_date)

df = df.loc[mask]
df

>**Prints Empty DataFrame Error**

>```
	1. open	2. high	3. low	4. close	5. adjusted close	6. volume	7. dividend amount	8. split coefficient
date	
```
- https://stackoverflow.com/questions/22898824/filtering-pandas-dataframes-on-dates

In [ ]:
print(data_daily.loc[datetime.date(year=2014,month=1,day=1):datetime.date(year=2015,month=2,day=1)])

In [ ]:
df.loc['2014-01-01':'2014-02-01']

Adding days to `start_date` and adding them to `date_list=[]`

In [ ]:
# Test example to create a list of possible deposit days
date_list = []

start_date = datetime.date(2018, 12, 31)
end_date = datetime.date(2020, 1, 4)
delta = datetime.timedelta(days=30)

while start_date <= end_date:
    print(start_date)
    start_date += delta
    date_list.append(start_date)
    
print(date_list)

## Date range filters

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html

In [10]:
print(data_daily)

             1. open   2. high  ...  7. dividend amount  8. split coefficient
date                            ...                                          
2020-09-21  325.7000  327.1300  ...              0.0000                   1.0
2020-09-18  335.3700  335.4900  ...              1.3392                   1.0
2020-09-17  333.5600  337.6996  ...              0.0000                   1.0
2020-09-16  341.5100  343.0600  ...              0.0000                   1.0
2020-09-15  341.1200  342.0200  ...              0.0000                   1.0
...              ...       ...  ...                 ...                   ...
1999-11-05  138.6250  139.1093  ...              0.0000                   1.0
1999-11-04  136.7500  137.3593  ...              0.0000                   1.0
1999-11-03  136.0000  136.3750  ...              0.0000                   1.0
1999-11-02  135.9687  137.2500  ...              0.0000                   1.0
1999-11-01  136.5000  137.0000  ...              0.0000         

In [16]:
data_daily.filter(like='1999', axis = 0)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
1999-12-31,146.8437,147.5000,146.2500,146.8750,99.4852,3172700.0,0.0000,1.0
1999-12-30,147.1250,147.5625,146.1875,146.6406,99.3265,3641300.0,0.0000,1.0
1999-12-29,146.3125,146.8125,145.3125,146.8125,99.4429,3001000.0,0.0000,1.0
1999-12-28,145.8750,146.5000,145.4843,146.1875,99.0195,4084500.0,0.0000,1.0
1999-12-27,146.5000,146.7812,145.0625,146.2812,99.0830,2691000.0,0.0000,1.0
1999-12-23,145.0156,146.4843,145.0000,146.4843,99.2206,5721700.0,0.0000,1.0
1999-12-22,143.6250,144.1875,142.9687,144.1875,97.6649,5377500.0,0.0000,1.0
1999-12-21,141.5937,144.0625,141.3437,143.8125,97.4109,7981300.0,0.0000,1.0
1999-12-20,142.5625,143.1875,141.0937,141.6562,95.9503,4608700.0,0.0000,1.0
